## Objective

The objective of this model is automate the classification of images of castings from a manufacturing process into one of two categories: 

* Defective
* OK

## Setup

The following libraries are required for the analysis:

In [ ]:
import pathlib

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from sklearn.metrics import confusion_matrix

import casting_defects_plots
import casting_defects_utils

The following parameters are used to configure the model.  

In [ ]:
BATCH_SIZE = 128
NO_EPOCHS = 5 
TRAIN_PROP = 0.05

The configuration settings control the following:

* `BATCH_SIZE`: The NN below is trained using [mini-batch gradient decent](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size).   This parameter sets the size of the mini-batch.
* `NO_EPOCHS`: An *epoch* refers to the model status after one entire cycle of the dataset has been used to update the weights.  The `NO_EPOCHS` parameter sets the number of complete cycles through the entire data set that the NN will be exposed to.
* `TRAIN_PROP`: Sets the proportion of data in the training set that will be used.  It should usually be 1.  However, during model development, it can be useful to adjust it to a small fraction e.g. 0.05 to run a quick 'test' of the setup.

## Data

The `ImageDataGenrator` class from `keras` is used to pre-process raw image data.  A large number of operations are available for [image augmentation](https://machinelearningmastery.com/image-augmentation-deep-learning-keras/).  In this present case, the only pre-processing that is performed is to normalise the images.  Additionally, a validation set is partitioned from the training data.

In [ ]:
# Paths to data.
input_dir = pathlib.Path.cwd().parent  / 'input'
data_sets = ('train', 'test')
train_dir, test_dir = [next(input_dir.rglob(ds)) for ds in data_sets]

# Generators.
# noinspection SpellCheckingInspection
trainval_datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2
)
# noinspection SpellCheckingInspection
test_datagen = ImageDataGenerator(
    rescale=1. / 255
)

Note that there are three data sets:

* **Training Set**. Used to compute the updates to the NN weights i.e. for training.
* **Validation Set**.  Held-out from the training process and used to compute an unbiased loss function score at the completion of each epoch.  This is used to keep track of how well training is progressing.
* **Test Set**.  Held-out from the training process all together.  The test set is used after all training has taken place to make an unbiased assessment of the final model.

Some examples of training images are shown below.

In [ ]:
SEED_VALUE = 123456
example_images = casting_defects_utils.generate_examples(
    no_examples=5,
    idg=trainval_datagen,
    directory=train_dir,
    target_size=(300, 300),
    color_mode='grayscale',
    seed=SEED_VALUE
)
casting_defects_plots.plot_examples(example_images, figsize=(8, 4))

The following code sets up objects for generating batches of images to feed to the NN.

In [ ]:
# Setup generators for train, validation and test data sets.
train_generator = trainval_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED_VALUE
)

validation_generator = trainval_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED_VALUE
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    color_mode='grayscale',
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

The generators above report the total number of images upon invocation.  Below, the total number of classes in the training set are summarised.

In [ ]:
cnt = dict(casting_defects_utils.summarise_classes(train_generator))
cnt = {k : (v, 100 * v / sum(cnt.values())) for k, v in cnt.items()}
msg = "{:9s}: {} observations ({:4.1f}%)"
for cls, (count, prop) in cnt.items():
    print(msg.format(cls, count, prop))
    

It is observed that the training example are fairly well balanced in terms of their class.

## Model Architecture

The model architecture is defined using the `Sequential` API in `keras`.  The architecture consists of 3 alternating `Conv2D` and `MaxPool2D` layers.  The convolutional layers respond to features in the images.  Examples of features may be may be horizontal lines, vertical lines, circles, etc.  In deep learning networks, the complexity of the features detected increases with the layer depth in the network.

The convolutional layers contain 32, 64 and 32 filters respectively.  During training, the weights will be updated such that each of the filters will respond to different features in the training set.  The training processes involves manipulating the NN weights such that the identified combination and layout of features (e.g. dark/light spots, jagged/smooth lines, variance/invariance) in a particular image will *activate* the most likely class to which the image belongs.

The pooling layers aggregate the outputs from the convolutions which has the advantage of a) reducing the number of weights required to train the network and b) combating over-fitting.

The final Flatten and Dense layers connect the output from the preceding layer to a single node.  This problem is one of *binary classification*.  The *sigmoid* activation function on the final node will vary between 0 and 1 and will model the probability of the image at the input to the NN corresponding to one class (1) or the other (0).

In [ ]:
# Create neural network architecture.
model = Sequential([
    Conv2D(32, kernel_size=3, input_shape=(300, 300, 1), activation='relu'),
    MaxPool2D(4),
    Conv2D(64, kernel_size=3, input_shape=(300, 300, 1), activation='relu'),
    MaxPool2D(4),
    Conv2D(32, kernel_size=3, input_shape=(300, 300, 1), activation='relu'),
    Flatten(),
    Dense(1, activation='sigmoid')
])

model.summary()

The model summary shows the number of parameters (weights).  It is useful to consider this in the context of the size of the training set i.e. in general, the more parameters that there are, the more training data required.

## Model Compilation

Before the model can be trained, it must be *compiled*.  Firstly, an optimiser is specified.  In this case, the **adam** optimiser is chosen which, in summary, is [gradient decent](https://machinelearningmastery.com/gradient-descent-for-machine-learning/) featuring a learning rate that adjusts as training progresses. The name 'adam' is derived from 'Adaptive Moment Estimation' and it is often recommended as a robust optimisation algorithm.

The loss function is set to 'binary cross-entropy' which is appropriate for the current class of problem i.e. binary categorisation.  The binary cross-entropy quantifies how correct the model is.  Additionally, the metric, 'accuracy' is specified.  This is not used to assess the model but it is used in the output since it is easier to interpret than binary cross entropy.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

## Model Training 

The model is now ready to be trained.  The train steps per epoch are set such that most images in the training set will be used for training in each epoch and each image will only be used once in an epoch.  The aim is to expose to the NN to every image once in each epoch but since the number of training examples may not be exactly divisible by the `BATCH_SIZE`, some images may not be included in a given epoch.

After training the model, *learning curves* are plotted.  They show the history of the loss function computed for the training and validation data at the end of each epoch.  The learning curves are useful for assessing the quality of the model.   Also, they are a useful tool for diagnosing issues such as over-fitting or under-fitting, whether the model might benefit from running more epochs or whether it may benefit from more data.  A 'good' training curve would show both the training and validation error converging, in a stable fashion, to a small and similar loss function value.

In [ ]:
# Train model.
train_steps, val_steps = [TRAIN_PROP * (gen.samples // BATCH_SIZE)
                          for gen in (train_generator, validation_generator)]
training = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=train_steps,
    validation_data=validation_generator,
    validation_steps=val_steps,
    epochs=NO_EPOCHS
)

## Evaluate model 

The final model is evaluated on the *test* set.  The accuracy is as follows:

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator)
print("Accuracy on test set: {:.3f}".format(test_acc))

Often, [confusion matrices](https://machinelearningmastery.com/confusion-matrix-machine-learning/) are a more appropriate method for assessing performance, especially for unbalanced classes.  The confusion matrix is as follows:

In [ ]:
# Calculate predicted probabilities, compute confusion matrix, plot.
pred_prob = model.predict_generator(test_generator)
pred_class = [1 if p > 0.5 else 0 for p in pred_prob]
true_class = test_generator.classes
labels = {v: k for k, v in test_generator.class_indices.items()}
labels = [labels[0], labels[1]]

cm = confusion_matrix(pred_class, true_class)

casting_defects_plots.plot_confusion_matrix(cm, labels=labels)